## Geo-Locations zu zusätzlichen Merkmalen transformieren

In [ ]:
# ! pip install -U pandas geopandas geopy folium scikit-learn

Wir arbeiten wieder mit Pandas, aber auch der Erweiterung GeoPandas, welche die Arbeit mit Koordinaten-basierten Daten vereinfacht. Zudem benötigen wir wieder SciKit-Learn, um ein ML-Modell zu trainieren und zu evaluieren.

In [ ]:
import pandas as pd
import geopandas as gpd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

In diesem Fall haben wir einen vorliegenden Datensatz als GeoJSON, welcher Koordinaten und einen Zielwert zur Vorhersage enthält.

Wenn wir nun ein ML-Modell mit sklearn trainieren wollen, müssten wir auf die X und Y Koordinaten zurückgreifen

In [ ]:
X = pd.DataFrame([data.geometry.y, data.geometry.x]).transpose() # Merkmale
y = data.y # Zielwert

X

In [ ]:
reg = GradientBoostingRegressor()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

reg = reg.fit(X_train, y_train)
reg.score(X_test, y_test)

Die Vorhersage anhand der Koordinaten läuft also nicht zu gut. Wir können versuchen, stattdessen die Distanz zur nächsten Haltestelle des MVV als Merkmal zu nutzen. 

Diese gibt es im Datensatz auf dem Open Data Portal: https://opendata.muenchen.de/dataset/haltestellenliste-mvv

Nun erstellen wir eine Funktion, die die Distanz zur nächsten Station (Luftlinie) berechnet und wenden diese auf unseren Datensatz an:

In [ ]:
def get_nearest_station(point):
    return round(stations.to_crs(epsg=25832).distance(point).min())

Wenn wir den Regressor nun mit dem neuen Merkmal nearest_station trainieren, erhalten wir einen weitaus besseren Score:

In [ ]:
X, y = data.nearest_station.array.reshape(-1, 1), data.y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

reg = reg.fit(X_train, y_train)
reg.score(X_test, y_test)